# 実験パイプラインの実行

これは実験パイプラインを実行するためのノートブックです。

## 1. 実験パイプライン情報の取得

- このステップでは、実行する実験パイプラインの定義情報を取得します。
- 以下のセルを実行してください。

In [ ]:
import json

# 作業ディレクトリに移動
%cd ~/experiments/

# 実験パイプライン定義ファイルの読み込み
pipeline = json.load(open("pipeline.json"))
print("実験パイプライン：" + str(pipeline))

## 2. 実験パッケージの最新化

In [ ]:
for experiment in pipeline:
    %cd $experiment
    !git pull
    %cd ../

## 3. 実験パイプラインの実行

- このステップでは、定義情報に基づいて実験パイプラインを実行します。
- 以下のセルを実行してください。

In [ ]:
%cd ~/experiments/

import papermill

outputs = []

for experiment in pipeline:
    print("\n============================================================ " + experiment)

    print("\n# 出力ディレクトリの初期化")
    cmd = "rm -rf " + experiment + "/output_data/*"
    print("$ " + cmd)
    !$cmd
    
    print("\n前実験の出力データを当実験の入力データとして複製")
    for output in outputs:
        cmd = "cp -rf " + output + "/output_data/* " + experiment + "/input_data/" 
        print("$ " + cmd)
        !$cmd

    print("\n実験再現のための環境構築")
    cmd = "pip install -r " + experiment + "/requirements.txt"
    print("$ " + cmd)
    !$cmd

    print("\n実験ソースコードの実行")
    cmd = experiment + "/source/"
    print("$ cd " + cmd)
    %cd $cmd

    papermill.execute_notebook("main.ipynb", "log.ipynb")
    
    cmd = "-"
    print("$ cd " + cmd)
    %cd $cmd

    print("\n次実験に備えて本実験の出力データを登録")
    outputs += [experiment]